In [72]:
%matplotlib inline

import os
import shutil
import time

from IPython.display import Image
import matplotlib.pyplot as plt

import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import DataLoader

In [73]:
# ??DataLoader

In [74]:
# Trainer parameters
print_freq_epochs = 1
use_cuda = True

# Dataset Parameters
batch_size = 200
load_size = 128
fine_size = 128
c = 3
data_mean = np.asarray([0.45834960097,0.44674252445,0.41352266842])

# Training parameters
architecture = 'resnet18'
lr = 0.005  # Can be large for resnet or if using batch norm
momentum = 0.9
weight_decay = 1e-4
num_epochs = 1

In [75]:
%pwd

'/workspace/miniplaces/model/pytorch'

In [76]:
def construct_dataloader_disk():
    # Construct DataLoader
    opt_data_train = {
        #'data_h5': 'miniplaces_128_train.h5',
        'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        'data_list': '../../data/train.txt', # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': True
        }
    opt_data_val = {
        #'data_h5': 'miniplaces_128_val.h5',
        'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        'data_list': '../../data/val.txt',   # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': False
        }

    loader_train = DataLoader.DataLoaderDisk(**opt_data_train)
    loader_val = DataLoader.DataLoaderDisk(**opt_data_val)
    
    return (loader_train, loader_val)

def construct_dataloader_h5():
    # Construct DataLoader from an h5 file
    opt_data_train = {
        'data_h5': 'miniplaces_128_train.h5',
        #'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        #'data_list': '../../data/train.txt', # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': True
        }
    opt_data_val = {
        'data_h5': 'miniplaces_128_val.h5',
        #'data_root': '../../data/images/',   # MODIFY PATH ACCORDINGLY
        #'data_list': '../../data/val.txt',   # MODIFY PATH ACCORDINGLY
        'load_size': load_size,
        'fine_size': fine_size,
        'data_mean': data_mean,
        'randomize': False
        }

    loader_train = DataLoader.DataLoaderH5(**opt_data_train)
    loader_val = DataLoader.DataLoaderH5(**opt_data_val)
    
    return (loader_train, loader_val)

In [77]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def adjust_learning_rate(lr, optimizer, epoch):
    """Calculates a learning rate of the initial LR decayed by 10 every 30 epochs"""
    lr = lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [89]:
# train and validate methods adapted from https://github.com/pytorch/examples/blob/master/imagenet/main.py

def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i in range(train_loader.size()):
        input, target = train_loader.next_batch(batch_size)
        target = target.long()
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            target = target.cuda(async=True)
            input = input.cuda(async=True)
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)
        target_var = target_var.long()
        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq_epochs == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   epoch, i, train_loader.size(), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1, top5=top5))

def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i in range(val_loader.size()):
        input, target = val_loader.next_batch(batch_size)
        
        if use_cuda:
            target = target.cuda(async=True)
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq_epochs == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, val_loader.size(), batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))

    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

    return top1.avg

In [90]:
model = models.__dict__[architecture]()

criterion = nn.CrossEntropyLoss()
if use_cuda:
    model = model.cuda()
    criterion = criterion.cuda()
    

optimizer = torch.optim.SGD(model.parameters(), lr,
                            momentum=momentum,
                            weight_decay=weight_decay)


In [80]:
# train_loader, val_loader = construct_dataloader_disk()
train_loader, val_loader = construct_dataloader_h5()

# Images found: 100000
# Images found: 10000


In [81]:
# I probably want to create a sample

In [91]:
model.avgpool = nn.AdaptiveAvgPool2d(1)

In [92]:
#print(model)

In [ ]:
#for epoch in range(num_epochs):
epoch=1
lr = adjust_learning_rate(lr, optimizer, epoch)

# train for one epoch
train(train_loader, model, criterion, optimizer, epoch)

# evaluate on validation set
prec1 = validate(val_loader, model, criterion)

# remember best prec@1 and save checkpoint
is_best = prec1 > best_prec1
best_prec1 = max(prec1, best_prec1)
save_checkpoint({
    'epoch': epoch + 1,
    'arch': architecture,
    'state_dict': model.state_dict(),
    'best_prec1': best_prec1,
    'optimizer' : optimizer.state_dict(),
}, is_best)

Epoch: [1][0/100000]	Time 0.193 (0.193)	Data 0.134 (0.134)	Loss 7.1032 (7.1032)	Prec@1 0.000 (0.000)	Prec@5 0.500 (0.500)
Epoch: [1][1/100000]	Time 0.175 (0.184)	Data 0.119 (0.126)	Loss 7.0041 (7.0536)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.250)
Epoch: [1][2/100000]	Time 0.169 (0.179)	Data 0.116 (0.123)	Loss 6.9864 (7.0312)	Prec@1 0.000 (0.000)	Prec@5 0.000 (0.167)
Epoch: [1][3/100000]	Time 0.173 (0.177)	Data 0.119 (0.122)	Loss 6.8910 (6.9962)	Prec@1 0.000 (0.000)	Prec@5 0.500 (0.250)
Epoch: [1][4/100000]	Time 0.173 (0.177)	Data 0.120 (0.121)	Loss 6.8653 (6.9700)	Prec@1 0.000 (0.000)	Prec@5 2.500 (0.700)
Epoch: [1][5/100000]	Time 0.174 (0.176)	Data 0.120 (0.121)	Loss 6.8473 (6.9496)	Prec@1 0.500 (0.083)	Prec@5 2.000 (0.917)
Epoch: [1][6/100000]	Time 0.174 (0.176)	Data 0.120 (0.121)	Loss 6.7794 (6.9252)	Prec@1 0.000 (0.071)	Prec@5 2.000 (1.071)
Epoch: [1][7/100000]	Time 0.172 (0.175)	Data 0.119 (0.121)	Loss 6.7526 (6.9037)	Prec@1 0.500 (0.125)	Prec@5 4.000 (1.438)
Epoch: [1][8/100000]	Tim

Epoch: [1][68/100000]	Time 0.189 (0.178)	Data 0.128 (0.123)	Loss 4.4716 (5.2700)	Prec@1 4.000 (1.826)	Prec@5 15.000 (8.333)
Epoch: [1][69/100000]	Time 0.187 (0.178)	Data 0.133 (0.123)	Loss 4.4483 (5.2583)	Prec@1 2.500 (1.836)	Prec@5 14.500 (8.421)
Epoch: [1][70/100000]	Time 0.174 (0.178)	Data 0.120 (0.123)	Loss 4.5254 (5.2480)	Prec@1 3.500 (1.859)	Prec@5 13.500 (8.493)
Epoch: [1][71/100000]	Time 0.168 (0.178)	Data 0.115 (0.123)	Loss 4.4921 (5.2375)	Prec@1 4.000 (1.889)	Prec@5 14.500 (8.576)
Epoch: [1][72/100000]	Time 0.171 (0.178)	Data 0.117 (0.123)	Loss 4.5108 (5.2275)	Prec@1 1.500 (1.884)	Prec@5 10.500 (8.603)
Epoch: [1][73/100000]	Time 0.170 (0.178)	Data 0.116 (0.123)	Loss 4.4676 (5.2172)	Prec@1 1.500 (1.878)	Prec@5 13.500 (8.669)
Epoch: [1][74/100000]	Time 0.172 (0.178)	Data 0.116 (0.123)	Loss 4.4050 (5.2064)	Prec@1 3.500 (1.900)	Prec@5 16.500 (8.773)
Epoch: [1][75/100000]	Time 0.167 (0.177)	Data 0.113 (0.123)	Loss 4.4540 (5.1965)	Prec@1 3.500 (1.921)	Prec@5 13.000 (8.829)
Epoch: [

Epoch: [1][134/100000]	Time 0.171 (0.176)	Data 0.117 (0.121)	Loss 4.2621 (4.8161)	Prec@1 5.000 (3.037)	Prec@5 18.500 (12.315)
Epoch: [1][135/100000]	Time 0.168 (0.176)	Data 0.114 (0.121)	Loss 4.2193 (4.8117)	Prec@1 5.500 (3.055)	Prec@5 18.000 (12.357)
Epoch: [1][136/100000]	Time 0.169 (0.176)	Data 0.116 (0.121)	Loss 4.2406 (4.8075)	Prec@1 3.500 (3.058)	Prec@5 16.500 (12.387)
Epoch: [1][137/100000]	Time 0.169 (0.176)	Data 0.113 (0.121)	Loss 4.1547 (4.8028)	Prec@1 6.000 (3.080)	Prec@5 24.000 (12.471)
Epoch: [1][138/100000]	Time 0.177 (0.176)	Data 0.123 (0.121)	Loss 4.1564 (4.7981)	Prec@1 4.000 (3.086)	Prec@5 22.000 (12.540)
Epoch: [1][139/100000]	Time 0.167 (0.176)	Data 0.114 (0.121)	Loss 4.2568 (4.7943)	Prec@1 6.500 (3.111)	Prec@5 20.000 (12.593)
Epoch: [1][140/100000]	Time 0.182 (0.176)	Data 0.128 (0.121)	Loss 4.0681 (4.7891)	Prec@1 8.500 (3.149)	Prec@5 24.000 (12.674)
Epoch: [1][141/100000]	Time 0.174 (0.176)	Data 0.121 (0.121)	Loss 4.1933 (4.7849)	Prec@1 6.000 (3.169)	Prec@5 19.500 (

Epoch: [1][200/100000]	Time 0.171 (0.176)	Data 0.117 (0.121)	Loss 3.9437 (4.5772)	Prec@1 10.000 (4.356)	Prec@5 25.500 (15.881)
Epoch: [1][201/100000]	Time 0.171 (0.176)	Data 0.118 (0.121)	Loss 4.0115 (4.5744)	Prec@1 6.500 (4.366)	Prec@5 25.500 (15.928)
Epoch: [1][202/100000]	Time 0.177 (0.176)	Data 0.124 (0.121)	Loss 3.9691 (4.5714)	Prec@1 6.500 (4.377)	Prec@5 24.500 (15.970)
Epoch: [1][203/100000]	Time 0.171 (0.176)	Data 0.118 (0.121)	Loss 3.9722 (4.5685)	Prec@1 8.000 (4.395)	Prec@5 30.500 (16.042)
Epoch: [1][204/100000]	Time 0.174 (0.176)	Data 0.121 (0.121)	Loss 3.9557 (4.5655)	Prec@1 6.500 (4.405)	Prec@5 26.500 (16.093)
Epoch: [1][205/100000]	Time 0.166 (0.176)	Data 0.112 (0.121)	Loss 3.9369 (4.5624)	Prec@1 9.500 (4.430)	Prec@5 30.000 (16.160)
Epoch: [1][206/100000]	Time 0.173 (0.176)	Data 0.119 (0.121)	Loss 4.0153 (4.5598)	Prec@1 5.000 (4.432)	Prec@5 28.000 (16.217)
Epoch: [1][207/100000]	Time 0.187 (0.176)	Data 0.133 (0.121)	Loss 4.0658 (4.5574)	Prec@1 6.000 (4.440)	Prec@5 25.000 

Epoch: [1][266/100000]	Time 0.171 (0.175)	Data 0.116 (0.121)	Loss 3.8041 (4.4192)	Prec@1 10.000 (5.391)	Prec@5 28.500 (18.820)
Epoch: [1][267/100000]	Time 0.169 (0.175)	Data 0.116 (0.121)	Loss 3.8040 (4.4169)	Prec@1 9.500 (5.407)	Prec@5 30.000 (18.862)
Epoch: [1][268/100000]	Time 0.168 (0.175)	Data 0.115 (0.121)	Loss 3.8104 (4.4146)	Prec@1 9.500 (5.422)	Prec@5 27.000 (18.892)
Epoch: [1][269/100000]	Time 0.190 (0.175)	Data 0.136 (0.121)	Loss 3.9614 (4.4129)	Prec@1 7.000 (5.428)	Prec@5 26.000 (18.919)
Epoch: [1][270/100000]	Time 0.177 (0.175)	Data 0.117 (0.121)	Loss 3.9435 (4.4112)	Prec@1 8.000 (5.437)	Prec@5 30.500 (18.961)
Epoch: [1][271/100000]	Time 0.177 (0.175)	Data 0.123 (0.121)	Loss 3.7093 (4.4086)	Prec@1 14.500 (5.471)	Prec@5 36.500 (19.026)
Epoch: [1][272/100000]	Time 0.184 (0.175)	Data 0.130 (0.121)	Loss 3.8607 (4.4066)	Prec@1 12.500 (5.496)	Prec@5 31.000 (19.070)
Epoch: [1][273/100000]	Time 0.175 (0.175)	Data 0.121 (0.121)	Loss 3.7773 (4.4043)	Prec@1 14.500 (5.529)	Prec@5 29.5

Epoch: [1][331/100000]	Time 0.177 (0.176)	Data 0.123 (0.121)	Loss 3.6332 (4.2994)	Prec@1 12.500 (6.482)	Prec@5 38.500 (21.407)
Epoch: [1][332/100000]	Time 0.169 (0.176)	Data 0.115 (0.121)	Loss 3.7502 (4.2977)	Prec@1 12.500 (6.500)	Prec@5 30.500 (21.434)
Epoch: [1][333/100000]	Time 0.168 (0.176)	Data 0.115 (0.121)	Loss 3.7525 (4.2961)	Prec@1 10.000 (6.510)	Prec@5 28.500 (21.455)
Epoch: [1][334/100000]	Time 0.168 (0.176)	Data 0.114 (0.121)	Loss 3.8071 (4.2946)	Prec@1 13.500 (6.531)	Prec@5 31.500 (21.485)
Epoch: [1][335/100000]	Time 0.170 (0.176)	Data 0.116 (0.121)	Loss 3.7155 (4.2929)	Prec@1 12.000 (6.548)	Prec@5 31.000 (21.513)
Epoch: [1][336/100000]	Time 0.181 (0.176)	Data 0.127 (0.121)	Loss 3.7003 (4.2912)	Prec@1 14.500 (6.571)	Prec@5 35.000 (21.553)
Epoch: [1][337/100000]	Time 0.173 (0.176)	Data 0.119 (0.121)	Loss 3.7087 (4.2894)	Prec@1 10.500 (6.583)	Prec@5 32.500 (21.586)
Epoch: [1][338/100000]	Time 0.170 (0.176)	Data 0.116 (0.121)	Loss 3.7085 (4.2877)	Prec@1 10.000 (6.593)	Prec@5 

Epoch: [1][396/100000]	Time 0.172 (0.176)	Data 0.118 (0.121)	Loss 3.5009 (4.2020)	Prec@1 13.000 (7.475)	Prec@5 39.500 (23.591)
Epoch: [1][397/100000]	Time 0.184 (0.176)	Data 0.131 (0.121)	Loss 3.5640 (4.2004)	Prec@1 12.000 (7.486)	Prec@5 33.000 (23.614)
Epoch: [1][398/100000]	Time 0.171 (0.176)	Data 0.117 (0.121)	Loss 3.5005 (4.1986)	Prec@1 15.000 (7.505)	Prec@5 40.500 (23.657)
Epoch: [1][399/100000]	Time 0.173 (0.176)	Data 0.119 (0.121)	Loss 3.5274 (4.1969)	Prec@1 14.500 (7.522)	Prec@5 35.500 (23.686)
Epoch: [1][400/100000]	Time 0.189 (0.176)	Data 0.135 (0.121)	Loss 3.5843 (4.1954)	Prec@1 13.500 (7.537)	Prec@5 37.500 (23.721)
Epoch: [1][401/100000]	Time 0.171 (0.176)	Data 0.117 (0.121)	Loss 3.5406 (4.1938)	Prec@1 19.000 (7.566)	Prec@5 39.000 (23.759)
Epoch: [1][402/100000]	Time 0.178 (0.176)	Data 0.124 (0.121)	Loss 3.6720 (4.1925)	Prec@1 10.000 (7.572)	Prec@5 33.500 (23.783)
Epoch: [1][403/100000]	Time 0.174 (0.176)	Data 0.120 (0.121)	Loss 3.5819 (4.1910)	Prec@1 10.500 (7.579)	Prec@5 

Epoch: [1][462/100000]	Time 0.170 (0.175)	Data 0.116 (0.121)	Loss 3.5705 (4.1165)	Prec@1 11.000 (8.359)	Prec@5 40.500 (25.616)
Epoch: [1][463/100000]	Time 0.172 (0.175)	Data 0.118 (0.121)	Loss 3.4998 (4.1152)	Prec@1 16.500 (8.376)	Prec@5 46.500 (25.661)
Epoch: [1][464/100000]	Time 0.169 (0.175)	Data 0.115 (0.121)	Loss 3.7457 (4.1144)	Prec@1 11.500 (8.383)	Prec@5 34.500 (25.680)
Epoch: [1][465/100000]	Time 0.169 (0.175)	Data 0.115 (0.121)	Loss 3.6046 (4.1133)	Prec@1 14.500 (8.396)	Prec@5 39.500 (25.709)
Epoch: [1][466/100000]	Time 0.168 (0.175)	Data 0.115 (0.121)	Loss 3.6111 (4.1122)	Prec@1 14.000 (8.408)	Prec@5 41.500 (25.743)
Epoch: [1][467/100000]	Time 0.177 (0.175)	Data 0.123 (0.121)	Loss 3.5904 (4.1111)	Prec@1 11.500 (8.415)	Prec@5 37.500 (25.768)
Epoch: [1][468/100000]	Time 0.170 (0.175)	Data 0.116 (0.121)	Loss 3.6186 (4.1100)	Prec@1 12.500 (8.423)	Prec@5 40.000 (25.799)
Epoch: [1][469/100000]	Time 0.171 (0.175)	Data 0.117 (0.121)	Loss 3.6321 (4.1090)	Prec@1 15.000 (8.437)	Prec@5 